In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import sys 
import datetime as dt
import time

LESSON_DATA_FOLDER = './data/'
#http://www.imdb.com/interfaces/

In [16]:
class SingletonDecorator:
    def __init__(self,klass):
        self.klass = klass
        self.instance = None
    def __call__(self,*args,**kwds):
        if self.instance == None:
            self.instance = self.klass(*args,**kwds)
        return self.instance

In [17]:
import ipyparallel
import pandas

class _ParallelUtil:
            
    def __init__(self):        
        self.__CLUSTER_REQUESTS = 0
        self.__CLUSTER_FAILURES = []      
        self.__CLUSTERS = None
        self.getCluster()
        
    @staticmethod
    def __PARALLEL_CLUSTERS(throwError = False):
        try:
            PARALLEL_CONFIG = pandas.read_table(''.join([LESSON_DATA_FOLDER, 'parallel.config.tsv']))
            PARALLEL_CONFIG.replace(to_replace={'None':None},method='pad', inplace=True)
            default_config = PARALLEL_CONFIG[PARALLEL_CONFIG['default'] == True].to_dict('records')[0]
            default_config


            CLUSTERs = ipyparallel.Client(profile=default_config['profile'],sshserver=default_config['sshserver'],
                                          password=default_config['password'])
            print('PROFILE: {} || IDS: {} '.format(CLUSTERs.profile, CLUSTERs.ids))
            #print("IDs:", CLUSTER.ids) # Print process id numbers
    
            return CLUSTERs
        except Exception as ex:            
            print (ex)
            if(throwError) :
                raise
            else:
                pass
                return None
            
                
    def setImports(self):
        clusters = self.getCluster()

        dview = clusters[:]
        with dview.sync_imports():
            import pandas
            import numpy
            import datetime
            import time
                     
    def getCluster(self):
        if(self.__CLUSTERS == None):
            try:
                self.__CLUSTER_REQUESTS  = self.__CLUSTER_REQUESTS + 1
                self.__CLUSTERS = _ParallelUtil.__PARALLEL_CLUSTERS(True)
            except Exception as ex:   
                self.__CLUSTER_FAILURES.append(ex)
                pass
            
        return self.__CLUSTERS
    
    def info(self):
        print ('CLUSTER_REQUESTS: {}, CLUSTER_FAILURES: {}'.format(self.__CLUSTER_REQUESTS, self.__CLUSTER_FAILURES))

class ParallelUtil: pass
ParallelUtil = SingletonDecorator(_ParallelUtil)        

In [18]:
ParallelUtil().getCluster()
#clusterUtil.info()
#ParallelUtil.__PARALLEL_CLUSTERS(True)

Hub connection request timed out
Hub connection request timed out


In [19]:
import datetime 
import numpy

class Utils:
    
    @staticmethod
    def index_array(length):
        result = [x for x in range(length)]   
        return result
    
    @staticmethod
    def parse_date(date):
        if str(date) == '' or date == None:
            return None
        else:
            return datetime.strptime(date,'%Y-%m-%d')    
        
    @staticmethod
    def parse_int(i, defaultValue=None):    
       # print (defaultValue)
        if ( i == None or str(i) == '' or str(i) == 'NaN' or i == numpy.NaN) :
            return defaultValue   
        else:
            try:
                #print ('convert', int(i))
                return int(float(i))
            except:
                return i
            
    @staticmethod
    def parse_float(f):
        if str(f) == '' or f == None:
            return None
        else:
            try:
                return float(f)
            except:
                return f
            
    @staticmethod
    def parse_bool(boolean):
        if str(boolean) == '' or boolean == None:
            return None
        else:        
            return boolean =='True' 

    @staticmethod
    def split(data, delimiter=',' ):
        #print ('data: ', data)    
        if(data == '' or data == None or str(data) == None):
            return np.array([None])
        else:
            #return np.array(str(data).lower().split(delimiter))    
            lst = np.array(str(data).split(delimiter))       
            return lst    

    @staticmethod
    def lower(data):
        if(data == '' or data == None):
            return None
        else:
            return str(data).lower() 
    
    @staticmethod
    def upper(data):
        if(data == '' or data == None):
            return None
        else:
            return str(data).upper()     
    
    @staticmethod
    def replaceNaN(data):
        if(np.isnan(data)): 
            return None
        else:
            return data
    
    # Python does not have switch statment, rather use dict approach
    parser = {
            'int':parse_int,
            'date':parse_date,
            'bool':parse_bool
        }


In [20]:


FILE_MAPPINGS = {
'title.basics.tsv': {
      'index_col': None, 
      'dtype' : {'tconst':np.dtype('S'),
                 'titleType':np.dtype('S'), 
                 'primaryTitle':np.dtype('S'),
                 'originalTitle':np.dtype('S'),
                 'isAdult':np.dtype('S'),
                 'startYear':np.dtype('S'),
                 'endYear':np.dtype('S'),             
                 'runtimeMinutes':np.dtype('S'),
                 'genres':np.dtype('S')
                    },      
      'filePath':LESSON_DATA_FOLDER + 'title.basics.tsv',
      'to_replace':{
          'titleType':{'\\N':None},
          'primaryTitle':{'\\N':None},
          'originalTitle':{'\\N':None},
          'startYear':{'\\N':None},
          'endYear':{'\\N':None},
          'runtimeMinutes':{'\\N':None}
      },
     'true_values':['1'],
     'false_values':['0'],     
     'usecols':['tconst','titleType','primaryTitle','originalTitle','isAdult',
                'startYear','endYear','runtimeMinutes','genres'],
     'converters' : {
                 #'primaryTitle':[{'function':Utils.lower, 'args':None}],
                 'titleType':[{'function':Utils.lower, 'args':None}],
                 #'originalTitle':[{'function':Utils.lower, 'args':None}],
                 'isAdult':[{'function':Utils.parse_bool, 'args':None}],
                 'startYear':[{'function':Utils.parse_int, 'args':None}] ,
                 'endYear':[{'function':Utils.parse_int, 'args':None}]  ,
                 'runtimeMinutes':[{'function':Utils.parse_int, 'args':None}],
                 'genres': [{'function':Utils.split, 'args':(',',)}]
                 
                }
    },
'title.crew.tsv': {
      'index_col': None, 
      'dtype' : {'tconst':np.dtype('S'),'directors':np.dtype('S') ,'writers':np.dtype('S')  },
      'split' : ['directors','writers'],
      'filePath':LESSON_DATA_FOLDER + 'title.crew.tsv',
      'to_replace':{
          'directors':{'\\N':None},
          'writers':{'\\N':None},          
      },
     'true_values':None,
     'false_values':None,     
     'usecols': None,
     'converters' : {                 
                 'writers':[{'function':Utils.split, 'args':(',',)}],
                 'directors': [{'function':Utils.split, 'args':(',',)}]
                }
    }, 
'title.episode.tsv': {
      'index_col': None, 
      'dtype' : {'tconst':np.dtype('S'),
                 'parentTconst':np.dtype('S'),
                 'seasonNumber':np.dtype('S'),  
                 'episodeNumber':np.dtype('S')  
                },
      'split' :None,
      'filePath':LESSON_DATA_FOLDER + 'title.episode.tsv',
      'to_replace':{
          'seasonNumber':{'\\N':None},
          'episodeNumber':{'\\N':None},                   
      },
      'true_values':None,
      'false_values':None,      
      'usecols': None,
      'converters' : {                 
                 'seasonNumber':[{'function':Utils.parse_int, 'args':None}],
                 'episodeNumber': [{'function':Utils.parse_int, 'args':None}]
                }
    },
'title.principals.tsv': {
      'index_col': None, 
      'dtype' : {'tconst':np.dtype('S'),
                 'ordering':np.dtype('S'),
                 'nconst':np.dtype('S'),
                 'category':np.dtype('S'),
                 'job':np.dtype('S'),                 
                 'characters':np.dtype('S'),                 
                },
      'split' : None,
      'filePath':LESSON_DATA_FOLDER + 'title.principals.tsv',
      'to_replace':{
          'job':{'\\N':None},
          'characters':{'\\N':None},                   
      },
      'true_values':None,
      'false_values':None,
      'converters' : {                 
                 'ordering':[{'function':Utils.parse_int, 'args':None}]             
                },
      'usecols': None      
    },    
'title.ratings.tsv': {
      'index_col': None, 
      'dtype' : {'tconst':np.dtype('S'),'averageRating':np.float64 ,'numVotes':np.int32  },
      'split' :None,
      'filePath':LESSON_DATA_FOLDER + 'title.ratings.tsv',
      'to_replace':None,
      'true_values':None,
      'false_values':None,      
      'usecols': None,
      'converters' : {                 
                 'numVotes':[{'function':Utils.parse_int, 'args':None}]               
                }    
    },  
'name.basics.tsv': {
      'index_col': None, 
      'dtype' : {'nconst':np.dtype('S'),
                 'primaryName':np.dtype('S') ,
                 'birthYear':np.dtype('S')  ,
                 'deathYear':np.dtype('S'),
                 'primaryProfession':np.dtype('S'),
                 'knownForTitles':np.dtype('S')
                },      
      'filePath':LESSON_DATA_FOLDER + 'name.basics.tsv',
      'to_replace':{
          'primaryProfession':{'\\N':None},
          'knownForTitles':{'\\N':None},    
          'birthYear':{'\\N':0},    
          'deathYear':{'\\N':0},       
      },
      'true_values':None,
      'false_values':None,
      'usecols': None,    
      'converters' : {                 
             #'primaryName':[{'function':Utils.lower, 'args':None}],                 
             'birthYear':[{'function':Utils.parse_int, 'args':(0,)}],                 
             'deathYear':[{'function':Utils.parse_int, 'args':(0,)}],
             'primaryProfession' :[{'function':Utils.split, 'args':(',',)}],
             'knownForTitles':[{'function':Utils.split, 'args':(',',)}],          
            } 
    },    
}

In [21]:
import pandas

def getMapping(file):
    return FILE_MAPPINGS.get(file)

def callFunction(columnData, **funDict):
    
    converFuns = funDict[columnData.name] #[{'function':split, 'args':(',')}]
    
    if(converFuns != None):
        for funSpec in converFuns:
            params = funSpec['args']
            #print ('args: ', params, ' <> ', params == None)
            if(params == None):                
                columnData = columnData.apply(funSpec['function'] )
            else:            
                columnData = columnData.apply(funSpec['function'], args=params )
            
    return columnData

def readFile(file, nrows=None, cluster=None ):   
    print ('Start: ' + time.strftime("%Y-%m-%d %H:%M:%S"))
    mapping = getMapping(file)
    dtype = mapping['dtype']
    usecols = list(dtype.keys())
    if(cluster != None):
        dview = cluster[:]
        dview.scatter(
            "df", 
            pandas.read_table(mapping['filePath'], 
                           index_col=mapping['index_col'], 
                           dtype = dtype, 
                           #na_values = ['//N'],
                           true_values= mapping['true_values'],
                           false_values= mapping['false_values'],                       
                           usecols=usecols,
                           nrows =nrows 
                          )
        )
        df = pandas.concat([i for i in dview["df"]])
    else:
        df = pandas.read_table(mapping['filePath'], 
                           index_col=mapping['index_col'], 
                           dtype = dtype, 
                           #na_values = ['//N'],
                           true_values= mapping['true_values'],
                           false_values= mapping['false_values'],                       
                           usecols=usecols,
                           nrows =nrows 
                          )
        
    
    df.fillna(method='pad', inplace=True)
    if(mapping['to_replace']!= None):
        df.replace(to_replace=mapping['to_replace'],method='pad', inplace=True)
    
    converters = mapping['converters']
    if(converters!= None):
        cols = list(converters.keys())
        df[cols] = df[cols].apply(callFunction, **converters)
        
    print ('End: ' + time.strftime("%Y-%m-%d %H:%M:%S"))
    return df

In [35]:
names = readFile('name.basics.tsv', nrows=100, cluster=ParallelUtil().getCluster())
names['isAlive'] = names['deathYear'] == 0

def wikiLink(name):
    link = ''.join (['https://en.wikipedia.org/wiki/', ''.join ([n + '_' for n in name.split()])]).rstrip('_')    
    return link

names['wikipedia'] = names['primaryName'].apply(wikiLink)
names[['lastName', 'firstName']] = names['primaryName'].apply(lambda x: pd.Series(str(x).lower().split(' ', 1)))
                       
def age(data):
    if(data['deathYear'] == 0):
        return 2017 - data['birthYear']
    else:
        return data['deathYear'] - data['birthYear']
            
names['age'] = names[['deathYear', 'birthYear']].apply(age, axis = 1)

names = names.astype({
                    'nconst':np.dtype('S20'),
                     'primaryName':np.dtype('S200') ,
                     'birthYear':np.int64  ,
                     'deathYear':np.int64,
                     'primaryProfession':np.dtype('S200'),
                     #'wikipedia':np.dtype('S1000'),
                     'lastName':np.dtype('S100'),
                     'firstName':np.dtype('S100'),
                })


names.info()
#ddf = dd.from_pandas(names, npartitions=2)


Hub connection request timed out
Start: 2018-03-02 16:54:28
End: 2018-03-02 16:54:28


SystemError: <class 'UnicodeEncodeError'> returned a result with an error set

In [26]:
names.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,isAlive,wikipedia,lastName,firstName,age
0,nm0000001,Fred Astaire,1899,1987,"[soundtrack, actor, miscellaneous]","[tt0050419, tt0072308, tt0043044, tt0053137]",False,https://en.wikipedia.org/wiki/Fred_Astaire,fred,astaire,88
1,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0117057, tt0037382, tt0040506, tt0038355]",False,https://en.wikipedia.org/wiki/Lauren_Bacall,lauren,bacall,90
2,nm0000003,Brigitte Bardot,1934,0,"[actress, soundtrack, producer]","[tt0049189, tt0063715, tt0059956, tt0057345]",True,https://en.wikipedia.org/wiki/Brigitte_Bardot,brigitte,bardot,83
3,nm0000004,John Belushi,1949,1982,"[actor, writer, soundtrack]","[tt0072562, tt0080455, tt0078723, tt0077975]",False,https://en.wikipedia.org/wiki/John_Belushi,john,belushi,33
4,nm0000005,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0050986, tt0050976, tt0083922, tt0060827]",False,https://en.wikipedia.org/wiki/Ingmar_Bergman,ingmar,bergman,89


In [ ]:
'''
# title.crew.tsv => many not need this file as title.principal.tsv 
crew = readFile('title.crew.tsv', 10000)#.compute()
crew.info()
'''

In [ ]:
def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, df[lst_cols[0]].str.len())
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

In [ ]:
'''
title_writers = explode(crew, ['writers'], fill_value='')
title_writers.drop('directors', axis=1, inplace=True)
#title_writers.info()
title_writers.rename(columns={'writers': 'nconst'}, inplace=True)
title_writers.columns
'''

In [ ]:
'''
title_directors = explode(crew, ['directors'], fill_value='')
title_directors.drop('writers', axis=1, inplace=True)
#title_directors.info()

title_directors.rename(columns={'directors': 'nconst'}, inplace=True)
title_directors.columns
'''

In [ ]:
'''
title_writers = title_writers.merge(names, on=['nconst'])
title_writers.head()
'''

In [ ]:
'''
title_directors = title_directors.merge(names, on=['nconst'])
title_directors.head()
'''

In [ ]:
'''
#title_writers.replace(to_replace={'knownForTitles':{np.NaN:None}},inplace=True)
#title_grouped_directors = 
title_grouped_directors = title_directors.head(10000).groupby('tconst')#.apply(f)
title_grouped_directors
'''

In [ ]:
'''
    #@dview.parallel(block=True)
    def f(group):
        #print (len(group))

        #group = pandas.DataFrame(group.to_dict('dict'))

        group = group.set_index('tconst')    
        data = {'directors': group.to_json(orient='records')}
        #data = {'directors': group.to_dict('dict')}
        #print (group.index.values , directors)
        return pandas.DataFrame(data=data, index = group.index.values)  



#title_grouped_directors# = 
#title_grouped_directors.reset_index().drop(['level_1'], axis= 1)
#pd.DataFrame(title_grouped_directors).to_json(orient='records')
#title_grouped_directors.reset_index(level='tconst')
'''

In [ ]:
'''
print (time.strftime("%Y-%m-%d %H:%M:%S"))
clusters = PARALLEL_CLUSTERS()
dview = clusters[:]
dview.scatter("scview", title_grouped_directors.apply(f))
#dview['scview']
title_grouped_directors = pd.concat([i for i in dview["scview"]])
print (time.strftime("%Y-%m-%d %H:%M:%S"))
#title_grouped_directors = title_grouped_directors.reset_index().drop(['level_1'], axis= 1)
'''

In [ ]:
ratings = readFile('title.ratings.tsv')
ratings.head(5)

In [ ]:
titles = readFile('title.basics.tsv', 10000)
titles['wikipedia'] = titles['primaryTitle'].apply(wikiLink)
#titles.tail()
#titles[titles['primaryTitle'] != titles['originalTitle']][['wikipedia','originalTitle','primaryTitle']].to_dict('record')

In [ ]:
#titles = titles.merge(ratings, on=['tconst'])
#titles[titles['primaryTitle'] != titles['originalTitle']].head(5)


In [ ]:
titlesp = pandas.read_table('./data/title.principals.tsv',nrows =1000) 
titlesc = pandas.read_table('./data/title.crew.tsv',nrows =1000) 

titlesc[titlesc['writers'] != '\\N']

#titlesc


In [ ]:
titlesp[titlesp['tconst'] == 'tt0000076']

In [ ]:
#names.info()
#del names
'''Important '''
#names.loc[['nm0000001'],['primaryName','birthYear','deathYear','primaryProfession']].to_dict('index')


del titles

In [ ]:
s1 = pd.Series({'aa':0, 'ba':1,'ca':2})
s2 = pd.Series({'ab':3, 'bb':4,'cb':5})


df = pd.DataFrame (data={
    'a':[1,2],
    'x':[s1,s2],
    'y':[['aa','ba','ca'],['ab','bb','cb']],
    'z':['a,b,c', 'd,e,f']
})
#df.iloc[0:].to_json(orient='records')
split_data = df['z'].str.split(',').apply(pd.Series, 1).stack()
#df_new.index
split_data.index = split_data.index.droplevel(-1)
split_data
df = df.join (pd.DataFrame(split_data))
df = df.reset_index(drop=True)
df

In [ ]:
pd.DataFrame({'directors': '[{"tconst":"tt8046488","nconst":"nm9649918","primaryName":"ricky capo","birthYear":0,"deathYear":0,"knownForTitles":[null]}]'}, index=['tt8046488'])


In [ ]:
df = pd.DataFrame([1,3.09])
df[0] = df[0].apply(np.int)
df

In [ ]:
ticket_series = df['Ticket'].str.split(' ').apply(pd.Series, 1).stack()